<h1>2b. Machine Learning using tf.estimator </h1>

In this notebook, we will create a machine learning model using tf.estimator and evaluate its performance.  The dataset is rather small (7700 samples), so we can do it all in-memory.  We will also simply pass the raw data in as-is. 

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import shutil

print(tf.__version__)

/usr/local/envs/py3env/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


1.8.0


Read data created in the previous chapter.

In [2]:
# In CSV, label is the first column, after the features, followed by the key
CSV_COLUMNS = ['fare_amount', 'pickuplon','pickuplat','dropofflon','dropofflat','passengers', 'key']
FEATURES = CSV_COLUMNS[1:len(CSV_COLUMNS) - 1]
LABEL = CSV_COLUMNS[0]

df_train = pd.read_csv('./taxi-train.csv', header = None, names = CSV_COLUMNS)
df_valid = pd.read_csv('./taxi-valid.csv', header = None, names = CSV_COLUMNS)
df_test = pd.read_csv('./taxi-test.csv', header = None, names = CSV_COLUMNS)

In [3]:
df_train.head(n = 2)
#type(df_train[LABEL])  # pandas.core.series.Series

,fare_amount,pickuplon,pickuplat,dropofflon,dropofflat,passengers,key
0,12.0,-73.987625,40.750617,-73.971163,40.785180,1,0
1,4.5,-73.963620,40.774363,-73.953485,40.772665,1,1


<h2> Train and eval input functions to read from Pandas Dataframe </h2>

In [4]:
# TODO: Create an appropriate input_fn to read the training data
def make_train_input_fn(df, num_epochs):
  return tf.estimator.inputs.pandas_input_fn(
    x = df,
    y = df[LABEL],
    batch_size = 128,
    num_epochs = num_epochs,
    shuffle = True
  )

In [5]:
# TODO: Create an appropriate input_fn to read the validation data
def make_eval_input_fn(df):
  return tf.estimator.inputs.pandas_input_fn(
    x = df,
    y = df[LABEL],
    batch_size = 128,
    shuffle = False
  )

Our input function for predictions is the same except we don't provide a label

In [6]:
# TODO: Create an appropriate prediction_input_fn
def make_prediction_input_fn(df):
  return tf.estimator.inputs.pandas_input_fn(
    x = df,
    batch_size = 128,
    shuffle = False
  )

### Create feature columns for estimator

In [7]:
df_train.head(n = 2)

,fare_amount,pickuplon,pickuplat,dropofflon,dropofflat,passengers,key
0,12.0,-73.987625,40.750617,-73.971163,40.785180,1,0
1,4.5,-73.963620,40.774363,-73.953485,40.772665,1,1


In [8]:
# TODO: Create feature columns
featcols = [
  tf.feature_column.numeric_column("pickuplon"),
  tf.feature_column.numeric_column("pickuplat"),
  tf.feature_column.numeric_column("dropofflon"),
  tf.feature_column.numeric_column("dropofflat"),
  tf.feature_column.numeric_column("passengers")
  #tf.feature_column.numeric_column("key")
]

<h3> Linear Regression with tf.Estimator framework </h3>

In [9]:
tf.logging.set_verbosity(tf.logging.INFO)

OUTDIR = 'taxi_trained'
shutil.rmtree(OUTDIR, ignore_errors = True) # start fresh each time

# TODO: Train a linear regression model
model = tf.estimator.LinearRegressor(featcols, OUTDIR) #ADD CODE HERE

model.train(  #ADD CODE HERE
  make_train_input_fn(df_train, num_epochs = 10),
  max_steps = 100000
)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_steps': None, '_session_config': None, '_log_step_count_steps': 100, '_evaluation_master': '', '_keep_checkpoint_max': 5, '_task_id': 0, '_keep_checkpoint_every_n_hours': 10000, '_save_summary_steps': 100, '_service': None, '_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f7ddc5e1630>, '_train_distribute': None, '_task_type': 'worker', '_num_worker_replicas': 1, '_is_chief': True, '_model_dir': 'taxi_trained', '_global_id_in_cluster': 0, '_tf_random_seed': None, '_master': ''}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into taxi_trained/model.ckpt.
INFO:tensorflow:step = 1, loss = 202

In [10]:
## start tensorboard:
from google.datalab.ml import TensorBoard as tb

tb.start(OUTDIR)

TensorBoard was started successfully with pid 9855. Click here to access it.

9855

Evaluate on the validation data (we should defer using the test data to after we have selected a final model).

In [11]:
def print_rmse(model, df):
  metrics = model.evaluate(input_fn = make_eval_input_fn(df))
  #print('RMSE on dataset = {}'.format(name, np.sqrt(metrics['average_loss'])))
  print('RMSE on dataset = {}'.format(np.sqrt(metrics['average_loss'])))
print_rmse(model, df_valid)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-01-30-15:19:24
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from taxi_trained/model.ckpt-608
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-01-30-15:19:24
INFO:tensorflow:Saving dict for global step 608: average_loss = 108.94069, global_step = 608, loss = 12956.161
RMSE on dataset = 10.43746566772461


This is nowhere near our benchmark (RMSE of $6 or so on this data), but it serves to demonstrate what TensorFlow code looks like.  Let's use this model for prediction.

In [12]:
# TODO: Predict from the estimator model we trained using test dataset
pred_iter = model.predict(make_prediction_input_fn(df_test))
for i in range(20):
  print(next(pred_iter))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from taxi_trained/model.ckpt-608
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
{'predictions': array([11.284742], dtype=float32)}
{'predictions': array([11.2821045], dtype=float32)}
{'predictions': array([11.282997], dtype=float32)}
{'predictions': array([11.28046], dtype=float32)}
{'predictions': array([11.28447], dtype=float32)}
{'predictions': array([11.284243], dtype=float32)}
{'predictions': array([11.282774], dtype=float32)}
{'predictions': array([11.282806], dtype=float32)}
{'predictions': array([11.284632], dtype=float32)}
{'predictions': array([11.282406], dtype=float32)}
{'predictions': array([11.284742], dtype=float32)}
{'predictions': array([11.284914], dtype=float32)}
{'predictions': array([11.277954], dtype=float32)}
{'predictions': array([11.282011], dtype=float32)}
{'predictions': array([1

This explains why the RMSE was so high -- the model essentially predicts the same amount for every trip.  Would a more complex model help? Let's try using a deep neural network.  The code to do this is quite straightforward as well.

<h3> Deep Neural Network regression </h3>

In [13]:
# TODO: Copy your LinearRegressor estimator and replace with DNNRegressor. 
# Remember to add a list of hidden units i.e. [32, 8, 2]

tf.logging.set_verbosity(tf.logging.INFO)

OUTDIR = 'taxi_trained'
shutil.rmtree(OUTDIR, ignore_errors = True) # start fresh each time

# TODO: Train a linear regression model
model = tf.estimator.DNNRegressor(feature_columns = featcols, hidden_units = [32, 8, 2], model_dir = OUTDIR) #ADD CODE HERE

model.train(  #ADD CODE HERE
  make_train_input_fn(df_train, num_epochs = 30),
  max_steps = 100000
)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_steps': None, '_session_config': None, '_log_step_count_steps': 100, '_evaluation_master': '', '_keep_checkpoint_max': 5, '_task_id': 0, '_keep_checkpoint_every_n_hours': 10000, '_save_summary_steps': 100, '_service': None, '_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f7db7385518>, '_train_distribute': None, '_task_type': 'worker', '_num_worker_replicas': 1, '_is_chief': True, '_model_dir': 'taxi_trained', '_global_id_in_cluster': 0, '_tf_random_seed': None, '_master': ''}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into taxi_trained/model.ckpt.
INFO:tensorflow:step = 1, loss = 250

In [14]:
def print_rmse(model, df):
  metrics = model.evaluate(input_fn = make_eval_input_fn(df))
  #print('RMSE on dataset = {}'.format(name, np.sqrt(metrics['average_loss'])))
  print('RMSE on dataset = {}'.format(np.sqrt(metrics['average_loss'])))
print_rmse(model, df_valid)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-01-30-15:19:33
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from taxi_trained/model.ckpt-1822
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-01-30-15:19:34
INFO:tensorflow:Saving dict for global step 1822: average_loss = 169.29991, global_step = 1822, loss = 20134.596
RMSE on dataset = 13.011529922485352


In [15]:
pred_iter = model.predict(make_prediction_input_fn(df_test))
for i in range(20):
  print(next(pred_iter))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from taxi_trained/model.ckpt-1822
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
{'predictions': array([3.8869517], dtype=float32)}
{'predictions': array([3.8869517], dtype=float32)}
{'predictions': array([3.8869517], dtype=float32)}
{'predictions': array([3.8869517], dtype=float32)}
{'predictions': array([3.8869517], dtype=float32)}
{'predictions': array([3.8869517], dtype=float32)}
{'predictions': array([3.8869517], dtype=float32)}
{'predictions': array([3.8869517], dtype=float32)}
{'predictions': array([3.8869517], dtype=float32)}
{'predictions': array([3.8869517], dtype=float32)}
{'predictions': array([3.8869517], dtype=float32)}
{'predictions': array([3.8869517], dtype=float32)}
{'predictions': array([3.8869517], dtype=float32)}
{'predictions': array([3.8869517], dtype=float32)}
{'predictions': array(

We are not beating our benchmark with either model ... what's up?  Well, we may be using TensorFlow for Machine Learning, but we are not yet using it well.  That's what the rest of this course is about!

But, for the record, let's say we had to choose between the two models. We'd choose the one with the lower validation error. Finally, we'd measure the RMSE on the test data with this chosen model.

<h2> Benchmark dataset </h2>

Let's do this on the benchmark dataset.

In [16]:
import google.datalab.bigquery as bq
import numpy as np
import pandas as pd

def create_query(phase, EVERY_N):
  """
  phase: 1 = train 2 = valid
  """
  base_query = """
SELECT
  (tolls_amount + fare_amount) AS fare_amount,
  EXTRACT(DAYOFWEEK FROM pickup_datetime) * 1.0 AS dayofweek,
  EXTRACT(HOUR FROM pickup_datetime) * 1.0 AS hourofday,
  pickup_longitude AS pickuplon,
  pickup_latitude AS pickuplat,
  dropoff_longitude AS dropofflon,
  dropoff_latitude AS dropofflat,
  passenger_count * 1.0 AS passengers,
  CONCAT(CAST(pickup_datetime AS STRING), CAST(pickup_longitude AS STRING), CAST(pickup_latitude AS STRING), CAST(dropoff_latitude AS STRING), CAST(dropoff_longitude AS STRING)) AS key
FROM
  `nyc-tlc.yellow.trips`
WHERE
  trip_distance > 0
  AND fare_amount >= 2.5
  AND pickup_longitude > -78
  AND pickup_longitude < -70
  AND dropoff_longitude > -78
  AND dropoff_longitude < -70
  AND pickup_latitude > 37
  AND pickup_latitude < 45
  AND dropoff_latitude > 37
  AND dropoff_latitude < 45
  AND passenger_count > 0
  """

  if EVERY_N == None:
    if phase < 2:
      # Training
      query = "{0} AND MOD(ABS(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING))), 4) < 2".format(base_query)
    else:
      # Validation
      query = "{0} AND MOD(ABS(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING))), 4) = {1}".format(base_query, phase)
  else:
    query = "{0} AND MOD(ABS(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING))), {1}) = {2}".format(base_query, EVERY_N, phase)
    
  return query

query = create_query(2, 100000)
df = bq.Query(query).execute().result().to_dataframe()
df.head(n = 2)

,fare_amount,dayofweek,hourofday,pickuplon,pickuplat,dropofflon,dropofflat,passengers,key
0,20.83,2.0,0.0,-73.942335,40.788055,-73.912777,40.766587,2.0,2014-08-11 00:45:00+00-73.94233540.78805540.76...
1,10.00,2.0,0.0,-73.954365,40.770057,-73.975082,40.789892,1.0,2014-08-11 00:45:00+00-73.95436540.77005740.78...


In [17]:
print_rmse(model, df)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-01-30-15:19:39
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from taxi_trained/model.ckpt-1822
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-01-30-15:19:39
INFO:tensorflow:Saving dict for global step 1822: average_loss = 142.8552, global_step = 1822, loss = 18150.914
RMSE on dataset = 11.952204704284668


RMSE on benchmark dataset is <b>9.41</b> (your results will vary because of random seeds).

This is not only way more than our original benchmark of 6.00, but it doesn't even beat our distance-based rule's RMSE of 8.02.

Fear not -- you have learned how to write a TensorFlow model, but not to do all the things that you will have to do to your ML model performant. We will do this in the next chapters. In this chapter though, we will get our TensorFlow model ready for these improvements.

In a software sense, the rest of the labs in this chapter will be about refactoring the code so that we can improve it.

## Challenge Exercise

Create a neural network that is capable of finding the volume of a cylinder given the radius of its base (r) and its height (h). Assume that the radius and height of the cylinder are both in the range 0.5 to 2.0. Simulate the necessary training dataset.
<p>
Hint (highlight to see):
<p style='color:white'>
The input features will be r and h and the label will be $\pi r^2 h$
Create random values for r and h and compute V.
Your dataset will consist of r, h and V.
Then, use a DNN regressor.
Make sure to generate enough data.
</p>

In [18]:
import math

def gen_data(n):
  r = np.random.uniform(.5, 2, n)
  h = np.random.uniform(.5, 2, n)
  v = r ** 2 * h * math.pi
  dat = pd.DataFrame({
    'r': r,
    'h': h,
    'v': v
  })
  return dat


dat = gen_data(10000)
dat_eval = gen_data(1000)
dat_test = gen_data(1000)

dat.head(n = 2)

,h,r,v
0,0.901820,0.571924,0.926715
1,0.991034,0.551736,0.947767


In [19]:
FEATURES = ['h', 'r']
LABEL = 'v'

featcols = [
  tf.feature_column.numeric_column("h"),
  tf.feature_column.numeric_column("r")
]

def make_train_input_fn(df, num_epochs):
  return tf.estimator.inputs.pandas_input_fn(
    x = df,
    y = df[LABEL],
    batch_size = 128,
    num_epochs = num_epochs,
    shuffle = True
  )

def make_eval_input_fn(df):
  return tf.estimator.inputs.pandas_input_fn(
    x = df,
    y = df[LABEL],
    batch_size = 128,
    shuffle = False
  )

def make_prediction_input_fn(df):
  return tf.estimator.inputs.pandas_input_fn(
    x = df,
    batch_size = 128,
    shuffle = False
  )

tf.logging.set_verbosity(tf.logging.INFO)
#tf.logging.set_verbosity(tf.logging.WARN)

OUTDIR = 'taxi_trained'
shutil.rmtree(OUTDIR, ignore_errors = True) # start fresh each time

# TODO: Train a linear regression model
#model = tf.estimator.LinearRegressor(featcols, OUTDIR) #ADD CODE HERE
model = tf.estimator.DNNRegressor(feature_columns = featcols, hidden_units = [4, 8, 4], model_dir = OUTDIR) #ADD CODE HERE

model.train(  #ADD CODE HERE
  make_train_input_fn(dat, num_epochs = 10), 
  max_steps = 100000
)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_steps': None, '_session_config': None, '_log_step_count_steps': 100, '_evaluation_master': '', '_keep_checkpoint_max': 5, '_task_id': 0, '_keep_checkpoint_every_n_hours': 10000, '_save_summary_steps': 100, '_service': None, '_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f7db7086ac8>, '_train_distribute': None, '_task_type': 'worker', '_num_worker_replicas': 1, '_is_chief': True, '_model_dir': 'taxi_trained', '_global_id_in_cluster': 0, '_tf_random_seed': None, '_master': ''}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into taxi_trained/model.ckpt.
INFO:tensorflow:step = 1, loss = 754

In [20]:
## RMSE:
def print_rmse(model, df):
  metrics = model.evaluate(input_fn = make_eval_input_fn(df))
  #print('RMSE on dataset = {}'.format(name, np.sqrt(metrics['average_loss'])))
  print('RMSE on dataset = {}'.format(np.sqrt(metrics['average_loss'])))
print_rmse(model, dat_eval)


INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-01-30-15:19:43
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from taxi_trained/model.ckpt-782
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-01-30-15:19:43
INFO:tensorflow:Saving dict for global step 782: average_loss = 2.6818922, global_step = 782, loss = 335.2365
RMSE on dataset = 1.637648344039917


In [21]:
## predict some values and get correlation:
pred_iter = model.predict(make_prediction_input_fn(dat_eval))
dat_pred = pd.DataFrame(columns = ['v_true', 'v_pred'])
for i in range(20):
  dat_pred = dat_pred.append({
    'v_true' : dat_eval['v'][i],
    'v_pred' : next(pred_iter)['predictions'][0]
  }, ignore_index = True)
  #print(dat_eval['v'][i], next(pred_iter)['predictions'][0])
  
dat_pred.head(n = 2)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from taxi_trained/model.ckpt-782
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


,v_true,v_pred
0,4.086859,4.503190
1,15.182077,13.600115


In [22]:
dat_pred.corr()

,v_true,v_pred
v_true,1.000000,0.962237
v_pred,0.962237,1.000000


Copyright 2017 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License